In [10]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [11]:
path_energy = os.path.join("dataset", "energy-cleaned-dataset.csv")
df_energy = pd.read_csv(path_energy)

In [12]:
THEME = "plotly_dark"
type = "Country"
name = "Afghanistan"
global na_info

In [13]:
def df_energy_query(area_type: str = type, area_name: str = name) -> pd.DataFrame : 
	"""
	The function df_energy_query return a filtered dataframe of df_energy.
	It useful because the computation is done only one time.
	:param area_type: 
	:param area_name: 
	:return: 
	"""
	return df_energy.query(f"{area_type} == '{area_name}'")

In [14]:
def get_na_info(df_filtered : pd.DataFrame = df_energy_query(), area_type = type, area_name = name) : 
    # compute the values for the indicator
    NB_OF_NAN = df_filtered.isna().sum().sum()
    NB_OF_NAN_GLOBAL = df_energy.isna().sum().sum()
    NB_MEAN_NAN = (NB_OF_NAN_GLOBAL / len(df_energy[area_type].unique()))
    # compare the nb of nan for the given area_name to the mean of nan of other area_type
    
    return NB_OF_NAN, NB_OF_NAN_GLOBAL, NB_MEAN_NAN

na_info = get_na_info()

In [15]:
def heatmap_missing_values(area_type: str = type, area_name: str = name) -> go.Figure:
    """
    This function creat a complexe multi-figure within a heatmap of missing values for the given area_name for all features and an indicator trace.
    :param area_type: ["Entity", "Continent", "Region", "iso3"]
    :param area_name: ["France", "Europe", "Western Europe", "FRA", ...]
    :return: a figure object of the plotly lib (heatmap + indicator)
    """
    
    # The set of var to observe except the one used on other axis
    col = df_energy.columns.tolist()
    for x in ['Year', 'Country', 'Continent', 'Region', 'iso3']: col.remove(x)

    df_heat = df_energy_query(area_type, area_name)[col]  # make the df with for the given area_name
    df_heat_na = df_heat.isna()
    df_heat_na.replace({True: 1, False: 0},
                       inplace=True)  # We change the True to 1 because plotly can not interpret them

    

    df_heat_na['Year'] = df_energy['Year']  # add the year column to the df
    transposed_df = df_heat_na.groupby('Year').sum().T

    # testing after the upper calculation to be sure that the nb of missing val and shape is plausible
    assert transposed_df.sum().sum() == df_energy.query(f"{area_type} == '{area_name}'").isna().sum().sum(), "<-- The sum of nan is  not the same -->"
    assert transposed_df.shape[0] == len(col), "<-- The number of columns is not the same -->"
    assert transposed_df.shape[1] == len(df_energy['Year'].unique().tolist()), "<-- The number of rows is not the same -->"

    heatmap = go.Figure()
    heatmap.add_trace(go.Heatmap(
                z=transposed_df,
                x=df_energy['Year'].unique().tolist(),
                y=col,
    
                # Styling
                colorscale='Viridis',
                colorbar=dict(
                    title="nombre",
                    titleside="top"
                )
            )
        )

    return heatmap
    
heatmap_missing_values()

In [16]:
NB_OF_NAN, NB_OF_NAN_GLOBAL, NB_MEAN_NAN = na_info

indicator_na = go.Figure()


indicator_na.add_trace(go.Indicator(
        mode="number+delta",  # 'delta' mean the % btw the value and a ref
        value=NB_OF_NAN,  # nb of nan for the current country looked
        delta={'reference': int(NB_MEAN_NAN), 'relative': True, 'valueformat': '.2f', "suffix": "%"},
        # styling
        title={"text": f"NN of nan for {name}<br>"
                       "<span style='font-size:0.8em;color:gray'>"
                       f"compare to the mean of {type}</span>"
               }
    )
)
indicator_na.update_layout(
    height=250,
)
indicator_na.show()

In [17]:
NB_OF_NAN, NB_OF_NAN_GLOBAL, NB_MEAN_NAN = na_info

indicator_na = go.Figure()

indicator_na.add_trace(go.Indicator(
    mode="number", 
    value=NB_OF_NAN_GLOBAL, 
    title={"text": f"NN of nan for {...}"}
)
)
indicator_na.update_layout(
    height=250,
)
indicator_na.show()

In [18]:
# Copiez le dataframe original pour travailler sur une copie
df_normalized = df_energy.copy()

# FIXME : remplmacer uniquement les 0
df_normalized.replace({np.nan: 1, 0: 1}, inplace=True)

# Sélectionnez les colonnes contenant des variables quantitatives à normaliser
cols_to_normalize = df_normalized.columns.difference(['Country', 'Year', 'Continent', 'Region', 'iso3', 'Access to Electricity (%)', 'Low-Carbon Electricity (%)', 'Renewables (% Equivalent Primary Energy)'])  
# Exclure les colonnes non numériques et celle qui sont deja en %

# On teste s'il n'y a pas de valeur restante non numérique
df_normalized[cols_to_normalize].apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())
# ADDDOC : Sur ce qu'on fait
# Créez un masque pour exclure les années autres que 2000
mask_2000 = (df_normalized['Year'] == 2000)
df_normalized_2000 = df_normalized[mask_2000][cols_to_normalize].reindex(df_normalized.index, method='pad')
# TODO : Ajouter le param, fill_na 


# HELP : test
"""
Le but est d'avoir des nan en sortie pour pas tracer des lines à 0
Mais ne pas diviser par 0 ou nan
"""
df_normalized[cols_to_normalize] = ((df_normalized[cols_to_normalize] - df_normalized_2000[cols_to_normalize]) / df_normalized_2000) * 100


df_normalized.head(50)

,Country,Year,Access to Electricity (%),Renewable Electricity Capacity per Capita,Electricity from Fossil Fuels (TWh),Electricity from Nuclear (TWh),Electricity from Renewables (TWh),Low-Carbon Electricity (%),Primary Energy Consumption per Capita (kWh/person),CO2 Emissions (kt by country),Renewables (% Equivalent Primary Energy),GDP Growth,GDP per Capita,Continent,Region,iso3,Human Development Index
0,Afghanistan,2000,1.613591,0.000000,0.000000,0.0,0.000000,65.957440,0.000000,0.000000,1.000000,0.000000,0.000000,Asia,Southern Asia,AFG,0.000000
1,Afghanistan,2001,4.074574,-3.904555,-43.750000,0.0,61.290323,84.745766,-21.713184,-3.947368,1.000000,0.000000,0.000000,Asia,Southern Asia,AFG,0.597015
2,Afghanistan,2002,9.409158,-8.134490,-18.750000,0.0,80.645161,81.159424,-30.315347,35.526312,1.000000,0.000000,17842.657920,Asia,Southern Asia,AFG,8.059701
3,Afghanistan,2003,14.738506,-12.255965,93.750000,0.0,103.225806,67.021280,-24.001270,60.526320,1.000000,783.227781,18968.381430,Asia,Southern Asia,AFG,12.238806
4,Afghanistan,2004,20.064968,-15.943601,106.250000,0.0,80.645161,62.921350,-32.506693,35.526312,1.000000,41.411798,21038.207420,Asia,Southern Asia,AFG,17.014925
5,Afghanistan,2005,25.390894,-18.546638,112.500000,0.0,90.322581,63.440857,-16.697477,103.947362,1.000000,1022.971482,24103.131320,Asia,Southern Asia,AFG,19.402985
6,Afghanistan,2006,30.718690,-19.739696,25.000000,0.0,106.451613,76.190475,0.603474,131.578946,1.000000,435.740325,26273.360190,Asia,Southern Asia,AFG,22.089552
7,Afghanistan,2007,36.051010,-21.366594,25.000000,0.0,141.935484,78.947370,17.080623,132.894734,1.000000,1282.631955,35869.315790,Asia,Southern Asia,AFG,26.567164
8,Afghanistan,2008,42.400000,-18.763557,18.750000,0.0,74.193548,73.972600,100.873729,368.421045,1.000000,292.498382,36366.354200,Asia,Southern Asia,AFG,28.358209
9,Afghanistan,2009,46.740050,-18.655098,0.000000,0.0,151.612903,82.978720,222.228966,542.105278,1.000000,2039.052839,43626.874020,Asia,Southern Asia,AFG,31.343284


In [53]:
df_line = df_normalized.query("Continent == 'Asia'")
df_line = df_line.groupby("Year").sum()
df_line.head()

,Country,Access to Electricity (%),Renewable Electricity Capacity per Capita,Electricity from Fossil Fuels (TWh),Electricity from Nuclear (TWh),Electricity from Renewables (TWh),Low-Carbon Electricity (%),Primary Energy Consumption per Capita (kWh/person),CO2 Emissions (kt by country),Renewables (% Equivalent Primary Energy),GDP Growth,GDP per Capita,Continent,Region,iso3,Human Development Index
Year,,,,,,,,,,,,,,,,
2000,AfghanistanArmeniaAzerbaijanBahrainBangladeshB...,3180.621629,0.000000,0.000000,0.000000,0.000000,959.045557,0.000000,0.000000,123.311978,0.000000,0.000000,AsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAs...,Southern AsiaWestern AsiaWestern AsiaWestern A...,AFGARMAZEBHRBGDBTNKHMCHNCYPGEOINDIDNIRQISRJPNJ...,0.000000
2001,AfghanistanArmeniaAzerbaijanBahrainBangladeshB...,3199.861412,19.370629,52.693137,452.666901,-58.680386,962.619006,13.938637,81.209513,120.130584,-469.949407,3.193728,AsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAs...,Southern AsiaWestern AsiaWestern AsiaWestern A...,AFGARMAZEBHRBGDBTNKHMCHNCYPGEOINDIDNIRQISRJPNJ...,39.295250
2002,AfghanistanArmeniaAzerbaijanBahrainBangladeshB...,3262.120413,14.810462,188.055352,451.721324,303.141405,990.412210,67.130623,226.621266,121.334062,557.316895,18027.723909,AsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAs...,Southern AsiaWestern AsiaWestern AsiaWestern A...,AFGARMAZEBHRBGDBTNKHMCHNCYPGEOINDIDNIRQISRJPNJ...,88.821746
2003,AfghanistanArmeniaAzerbaijanBahrainBangladeshB...,3273.634908,109.821573,2126.329756,523.683454,696.312347,983.650914,232.005253,421.077897,124.593360,2521.080668,19645.495382,AsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAs...,Southern AsiaWestern AsiaWestern AsiaWestern A...,AFGARMAZEBHRBGDBTNKHMCHNCYPGEOINDIDNIRQISRJPNJ...,137.915988
2004,AfghanistanArmeniaAzerbaijanBahrainBangladeshB...,3305.253719,240.628523,787.800128,651.466924,775.912541,976.119030,364.072035,631.547505,125.466778,4244.503375,22439.494436,AsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAsiaAs...,Southern AsiaWestern AsiaWestern AsiaWestern A...,AFGARMAZEBHRBGDBTNKHMCHNCYPGEOINDIDNIRQISRJPNJ...,183.167169


In [61]:

fig = px.line(df_line, x=df_line.index, y=cols_to_normalize, markers=True, log_y=False)

fig.show()